In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math
import scipy 
from scipy import stats


In [2]:
stocks = pd.read_csv('E:/Projects/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
496,YUM
497,ZBH
498,ZBRA
499,ZION


In [3]:
from secrets1 import IEX_CLOUD_API_TOKEN


In [39]:
symbol ='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2412339153636,
 'week52high': 187.94,
 'week52low': 126.59,
 'week52highSplitAdjustOnly': 185.33,
 'week52lowSplitAdjustOnly': 126.46,
 'week52change': 0.2025261051686713,
 'sharesOutstanding': 16517215347,
 'float': 0,
 'avg10Volume': 120056168,
 'avg30Volume': 104125637,
 'day200MovingAvg': 173,
 'day50MovingAvg': 172.99,
 'employees': 150605,
 'ttmEPS': 6.3,
 'ttmDividendRate': 0.8892557207611561,
 'dividendYield': 0.006348873145825354,
 'nextDividendDate': '',
 'exDividendDate': '2022-04-28',
 'nextEarningsDate': '2022-07-25',
 'peRatio': 23.3135104655311,
 'beta': 1.305888883118191,
 'maxChangePercent': 57.5942550226711,
 'year5ChangePercent': 3.0963373351776893,
 'year2ChangePercent': 0.9273175359687189,
 'year1ChangePercent': 0.1754422233301232,
 'ytdChangePercent': -0.1804071920286127,
 'month6ChangePercent': -0.006807960139704386,
 'month3ChangePercent': -0.13380551321890247,
 'month1ChangePercent': -0.11743100670766764,
 'day30Change

In [5]:
#parsing
data['year1ChangePercent']

0.1714537359037051

In [6]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings=[]
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
# final_dataframe = pd.DataFrame(columns = my_columns)

my_columns = ['Ticker', 'Price','One Year Price Return','No. of shares to Buy']

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data['AAPL']['stats'])
    for symbol in symbol_string.split(','):
        final_dataframe  =final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns),
        ignore_index = True
        )
        
final_dataframe

,Ticker,Price,One Year Price Return,No. of shares to Buy
0,A,117.13,-0.133178,N/A
1,AAL,15.59,-0.260928,N/A
2,AAP,213.40,0.006101,N/A
3,AAPL,148.89,0.171378,N/A
4,ABBV,156.01,0.387252,N/A
...,...,...,...,...
496,YUM,115.50,-0.074329,N/A
497,ZBH,115.30,-0.319906,N/A
498,ZBRA,317.82,-0.357665,N/A
499,ZION,54.59,-0.065933,N/A


In [8]:
final_dataframe.sort_values('One Year Price Return', ascending = False, inplace = True) #without inplace keyword it wont modify the original df
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One Year Price Return,No. of shares to Buy
0,273,LB,80.19,2.312265,N/A
1,147,DVN,66.92,1.667105,N/A
2,355,OXY,59.99,1.464773,N/A
3,315,MRO,26.14,1.237893,N/A
4,109,COP,103.60,0.867175,N/A
5,42,APA,39.76,0.863239,N/A
6,89,CF,102.51,0.824361,N/A
7,297,MCHP,65.81,0.799682,N/A
8,312,MOS,60.68,0.704863,N/A
9,298,MCK,336.59,0.694665,N/A


In [9]:
#calculating number of shares to buy
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    
    try:
        val = float(portfolio_size)

    except ValueError:
        print('Enter a number')
        portfolio_size = input('Enter the size of your portfolio: ')
        
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 100000000
100000000


In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'No. of shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])
    
final_dataframe

E:\Anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One Year Price Return,No. of shares to Buy
0,273,LB,80.19,2.312265,24940
1,147,DVN,66.92,1.667105,29886
2,355,OXY,59.99,1.464773,33338
3,315,MRO,26.14,1.237893,76511
4,109,COP,103.60,0.867175,19305
5,42,APA,39.76,0.863239,50301
6,89,CF,102.51,0.824361,19510
7,297,MCHP,65.81,0.799682,30390
8,312,MOS,60.68,0.704863,32959
9,298,MCK,336.59,0.694665,5941


In [27]:
#better strategy
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   0,
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   0,
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   0,
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   0,
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   0,
                                                   0
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [28]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.43,0,-0.132566,0,-0.285598,0,-0.171485,0,-0.13461,0,0
1,AAL,16.17,0,-0.269991,0,-0.252452,0,-0.096587,0,-0.060306,0,0
2,AAP,217.40,0,0.005934,0,-0.153431,0,-0.093166,0,-0.094131,0,0
3,AAPL,145.11,0,0.175868,0,-0.006638,0,-0.136354,0,-0.119087,0,0
4,ABBV,161.38,0,0.388941,0,0.337324,0,0.082696,0,-0.099178,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,113.86,0,-0.072021,0,-0.118049,0,-0.106105,0,-0.081602,0,0
497,ZBH,114.80,0,-0.331898,0,-0.149268,0,-0.022711,0,-0.107945,0,0
498,ZBRA,322.67,0,-0.367608,0,-0.467663,0,-0.293843,0,-0.235475,0,0
499,ZION,55.76,0,-0.0672,0,-0.18857,0,-0.267441,0,-0.171584,0,0


In [29]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0
        
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
# for row in hqm_dataframe.index:
#     for time_period in time_periods:
#         change_col = f'{time_period} Price Return'
#         percentile_col = f'{time_period} Return Percentile'
#         hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

#         hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return']), hqm_dataframe.loc[row, f'{time_period} Price Return']/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      35.528942
1      14.770459
2      59.281437
3      84.830339
4      95.209581
         ...    
496    46.706587
497     9.780439
498     6.986028
499     47.50499
500    49.101796
Name: One-Year Return Percentile, Length: 501, dtype: object
0      18.163673
1      22.954092
2      44.111776
3       67.46507
4      96.007984
         ...    
496    51.896208
497     45.10978
498     2.794411
499    37.125749
500    21.756487
Name: Six-Month Return Percentile, Length: 501, dtype: object
0      28.343313
1      46.307385
2      47.904192
3      37.724551
4      85.628743
         ...    
496    44.510978
497    64.870259
498     6.187625
499     9.580838
500    21.556886
Name: Three-Month Return Percentile, Length: 501, dtype: object
0      27.944112
1      62.075848
2      46.906188
3      33.732535
4      44.510978
         ...    
496     52.49501
497    41.716567
498     5.389222
499    17.365269
500    17.964072
Name: One-Month Return Percentile, Length: 501, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.43,0,-0.132566,35.528942,-0.285598,18.163673,-0.171485,28.343313,-0.13461,27.944112,0
1,AAL,16.17,0,-0.269991,14.770459,-0.252452,22.954092,-0.096587,46.307385,-0.060306,62.075848,0
2,AAP,217.40,0,0.005934,59.281437,-0.153431,44.111776,-0.093166,47.904192,-0.094131,46.906188,0
3,AAPL,145.11,0,0.175868,84.830339,-0.006638,67.46507,-0.136354,37.724551,-0.119087,33.732535,0
4,ABBV,161.38,0,0.388941,95.209581,0.337324,96.007984,0.082696,85.628743,-0.099178,44.510978,0
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,113.86,0,-0.072021,46.706587,-0.118049,51.896208,-0.106105,44.510978,-0.081602,52.49501,0
497,ZBH,114.80,0,-0.331898,9.780439,-0.149268,45.10978,-0.022711,64.870259,-0.107945,41.716567,0
498,ZBRA,322.67,0,-0.367608,6.986028,-0.467663,2.794411,-0.293843,6.187625,-0.235475,5.389222,0
499,ZION,55.76,0,-0.0672,47.50499,-0.18857,37.125749,-0.267441,9.580838,-0.171584,17.365269,0


In [30]:
#calculating HQM(high quality momentum) Score
from statistics import mean
for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)

hqm_dataframe
    
    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.43,0,-0.132566,35.528942,-0.285598,18.163673,-0.171485,28.343313,-0.13461,27.944112,27.49501
1,AAL,16.17,0,-0.269991,14.770459,-0.252452,22.954092,-0.096587,46.307385,-0.060306,62.075848,36.526946
2,AAP,217.40,0,0.005934,59.281437,-0.153431,44.111776,-0.093166,47.904192,-0.094131,46.906188,49.550898
3,AAPL,145.11,0,0.175868,84.830339,-0.006638,67.46507,-0.136354,37.724551,-0.119087,33.732535,55.938124
4,ABBV,161.38,0,0.388941,95.209581,0.337324,96.007984,0.082696,85.628743,-0.099178,44.510978,80.339321
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,113.86,0,-0.072021,46.706587,-0.118049,51.896208,-0.106105,44.510978,-0.081602,52.49501,48.902196
497,ZBH,114.80,0,-0.331898,9.780439,-0.149268,45.10978,-0.022711,64.870259,-0.107945,41.716567,40.369261
498,ZBRA,322.67,0,-0.367608,6.986028,-0.467663,2.794411,-0.293843,6.187625,-0.235475,5.389222,5.339321
499,ZION,55.76,0,-0.0672,47.50499,-0.18857,37.125749,-0.267441,9.580838,-0.171584,17.365269,27.894212


In [31]:
#selecting 50 best momentum stocks
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VLO,121.96,0,0.658826,97.804391,0.628207,99.401198,0.324453,99.401198,0.208421,99.800399,99.101796
1,LB,81.75,0,2.300545,100.0,0.82605,99.800399,0.216036,97.40519,0.082563,97.804391,98.752495
2,DVN,64.78,0,1.718581,99.800399,0.569596,99.201597,0.222762,97.804391,0.080342,97.40519,98.552894
3,OXY,60.93,0,1.502191,99.600798,0.895328,100.0,0.419677,99.800399,0.038242,94.011976,98.353293
4,MCK,326.62,0,0.703396,98.203593,0.513447,98.602794,0.229502,98.203593,0.036055,93.413174,97.105788
5,MPC,97.17,0,0.628524,97.40519,0.425134,97.804391,0.147981,94.610778,0.085267,98.003992,96.956088
6,COG,22.79,0,0.351725,93.812375,0.189472,89.620758,0.262889,98.602794,0.250336,100.0,95.508982
7,UNM,34.66,0,0.228766,89.221557,0.363245,96.806387,0.208278,97.205589,0.099521,98.403194,95.409182
8,MRO,25.62,0,1.269138,99.401198,0.539055,99.001996,0.142579,93.812375,-0.00205,83.832335,94.011976
9,PXD,260.00,0,0.671842,98.003992,0.428534,98.003992,0.104145,88.423154,0.013736,89.021956,93.363273


In [32]:
#calculating number of shares to buy
portfolio_input()



Enter the size of your portfolio: 1000000


In [33]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])
    
hqm_dataframe
    

E:\Anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VLO,121.96,163,0.658826,97.804391,0.628207,99.401198,0.324453,99.401198,0.208421,99.800399,99.101796
1,LB,81.75,244,2.300545,100.0,0.82605,99.800399,0.216036,97.40519,0.082563,97.804391,98.752495
2,DVN,64.78,308,1.718581,99.800399,0.569596,99.201597,0.222762,97.804391,0.080342,97.40519,98.552894
3,OXY,60.93,328,1.502191,99.600798,0.895328,100.0,0.419677,99.800399,0.038242,94.011976,98.353293
4,MCK,326.62,61,0.703396,98.203593,0.513447,98.602794,0.229502,98.203593,0.036055,93.413174,97.105788
5,MPC,97.17,205,0.628524,97.40519,0.425134,97.804391,0.147981,94.610778,0.085267,98.003992,96.956088
6,COG,22.79,877,0.351725,93.812375,0.189472,89.620758,0.262889,98.602794,0.250336,100.0,95.508982
7,UNM,34.66,577,0.228766,89.221557,0.363245,96.806387,0.208278,97.205589,0.099521,98.403194,95.409182
8,MRO,25.62,780,1.269138,99.401198,0.539055,99.001996,0.142579,93.812375,-0.00205,83.832335,94.011976
9,PXD,260.00,76,0.671842,98.003992,0.428534,98.003992,0.104145,88.423154,0.013736,89.021956,93.363273


In [37]:
#saving in excel
writer = pd.ExcelWriter('E:/Projects/momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy' , index= False)